In [ ]:
def apriori(dataset, min_support=0.5, verbose=False):
  C1 = create_candidate(dataset)
  D = map(set, dataset)
  F1, support_data = support_prune(D, C1, min_support,verbose=False)
  F = [F1]    #list of frequent itemsets, initialized to frequent 1-itemsets
  k = 2   #itemset cardinality
  while (len(F[k-2])>0):
    Ck = apriori_gen(F[k-2], k)   #generate candidate itemset
    Fk, supK = support_prune(D, Ck, min_support)  #prune candidate itemset 
    support_data.update(supK)   #update the support counts to reflect pruning 
    F.append(Fk)  #add the pruned candidate itemsets to the list of frequent itemset 
    k += 1 

    if verbose:
      #print a list of the frequent itemsets 
      for kset in F:
        for item in kset:
          print(""\
                + "{"\
                +"".join(str(i) + "," for i in iter(item).rstrip(',')\
                + "}"\
                + ": sup = " + str(round(support_data[item], 3))))

In [ ]:
def create_candidates(dataset, verbose=False):
  c1 = []   #list of all items in the database of transactions 
  for transaction in dataset:
    for item in transaction:
      if not [item] in c1:
        c1.append([item])
  c1.sort()

  if verbose:
    #print a list of all the candidate items 
    print("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")
    #map c1 to a frozenset bec it will be the key of a dictionary 
    return map(frozenset, c1)

In [ ]:
def support_prune(dataset, candidates, min_support, verbose=False):
  sscnt = {}    #set for support counts 
  for tid in dataset:
    for can in candidates:
      if can.issubset(tid):
        sscnt.setdefault(can, 0)
        sscnt[can] += 1
  num_items = float(len(dataset))   #total number of transactions in the dataset 
  retlist = []  #array for unpruned itemsets 
  support_data = {}   #set for support data for corresponding itemsets 
  for key in sscnt:
    #calculate the support of itemset key 
    support = sscnt[key] /num_items
    if support >= min_support:
      retlist.insert(0, key)
    support_data[key] = support 

    #print a list of the pruned itemsets 
    if verbose:
      for kset in retlist:
        for item in kset:
          print("{" + str(item) + "}")
          print("")

          for key in sscnt:
            print("" \
                  + "{"\
                  + "".join([str(item) + "}"
                  + "}" \
                  + ": sup = " + str(support_data[key])]))
    return retlist, support_data

In [ ]:
def apriori_gen(freq_sets, k):
  retList = []  #list of merged frequent itemsets 
  lenLk = len(freq_sets)  #number of frequent itemsets 
  for i in range(i+1, lenLk):
    a = list(freq_sets[i])
    b = list(freq_sets[j])
    a.sort()
    b.sort()
    F1 = a[:k-2]  #first k-2 items of freq_sets[i]
    F2 = b[:k-2]  #first k-2 items of freq_sets[j]

    if F1 == F2:
      #if the first k-2 items are identical 
      #merge the freq itemsets 
      retList.append(freq_sets[i] | freq_sets[j])
    return retList

In [ ]:
def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
  m = len(H[0])
  if m == 1:
        Hmp1 = calc_confidence(freq_set, H, support_data, rules, min_confidence, verbose)
  if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1, support_data, rules, min_confidence, verbose)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence, verbose)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.5, verbose=False):
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:
            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if verbose:
                print("" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

In [ ]:
def generate_rules(F, support_data, min_confidence=0.5, verbose=True):
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([itemset]) for itemset in freq_set]
            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence, verbose)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, verbose)

    return rules

In [ ]:
import pprint

def load_dataset():
    """Loads an example of market basket transactions for testing purposes.

    Returns
    -------
    A list (database) of lists (transactions). Each element of a transaction 
    is an item.
    """
    return [['Bread', 'Milk'], 
            ['Bread', 'Diapers', 'Beer', 'Eggs'], 
            ['Milk', 'Diapers', 'Beer', 'Coke'], 
            ['Bread', 'Milk', 'Diapers', 'Beer'], 
            ['Bread', 'Milk', 'Diapers', 'Coke']]

dataset = load_dataset() # list of transactions; each transaction is a list of items
D = map(set, dataset) # set of transactions; each transaction is a list of items

pprint.pprint(dataset)

[['Bread', 'Milk'],
 ['Bread', 'Diapers', 'Beer', 'Eggs'],
 ['Milk', 'Diapers', 'Beer', 'Coke'],
 ['Bread', 'Milk', 'Diapers', 'Beer'],
 ['Bread', 'Milk', 'Diapers', 'Coke']]
